# 使用Encoder-Decoder LSTM模型进行径流预报
## 加载数据

In [1]:
from hydronetwork.data.camels_us import load_timeseries
from hydronetwork.dataset import split_timeseries

data = load_timeseries(gauge_id="13023000", unit="mm/day")
train_data, test_data = split_timeseries(data=data, split_list=[0.8, 0.2])

In [2]:
data.columns

Index(['prcp(mm/day)', 'srad(W/m2)', 'tmax(C)', 'tmin(C)', 'vp(Pa)',
       'streamflow'],
      dtype='object', name='timeseries_type')

In [2]:
from hydronetwork.dataset import get_dataset

lookback = 365
horizon = 7
batchsize = 1

train_dataset = get_dataset(timeseries=train_data,
                            lookback=lookback,
                            horizon=horizon,
                            target="streamflow",
                            batch_size=batchsize,
                            # features_lookback=['streamflow'],
                            features_bidirectional=['prcp(mm/day)', 'srad(W/m2)', 'tmax(C)', 'tmin(C)', 'vp(Pa)'],
                            )
test_dataset = get_dataset(timeseries=test_data,
                           lookback=lookback,
                           horizon=horizon,
                           target="streamflow",
                           batch_size=batchsize,
                            # features_lookback=['streamflow'],
                           features_bidirectional=['prcp(mm/day)', 'srad(W/m2)', 'tmax(C)', 'tmin(C)', 'vp(Pa)'],
                           )

In [4]:
# 测试输出的数据有没有问题
for x, y in train_dataset:
    display(x)
    display(y)
    break

array([[[  4.65, 149.09,  -0.66, -12.11, 242.32],
        [  2.56, 158.5 ,  -0.77, -11.25, 263.73],
        [  1.44, 212.14,  -2.64, -15.04, 194.23],
        ...,
        [  4.52, 131.7 ,   0.63,  -9.24, 305.99],
        [  0.  , 154.66,   1.01,  -7.38, 354.42],
        [  0.  , 273.88,   0.47, -17.37, 160.82]]])

array([[0.50534725, 0.50534725, 0.48429112, 0.47376305, 0.45270691,
        0.44217884, 0.44217884]])

## 加载模型

In [15]:
from hydronetwork.model import encoder_decoder_lstm
from hydronetwork.train import WarmupExponentialDecay, callback_for_features_selection
from hydronetwork.evaluate.metric import rmse, nse
from keras.src.optimizers import AdamW
                                                                      
model = encoder_decoder_lstm(features_type=["bidirectional",], lookback=lookback, horizon=horizon)
model.compile(optimizer=AdamW(learning_rate=WarmupExponentialDecay(dataset_length=len(train_dataset),
                                                                   initial_learning_rate=1e-2,
                                                                   decay_rate=0.9,
                                                                   warmup_epochs=10)),
              loss=nse,
              metrics=[rmse],
              )

## 训练模型

In [ ]:
model.fit(train_dataset,
          epochs=1000,
          verbose=2,
          shuffle=True,
          validation_data=test_dataset, ) 

Epoch 1/1000
3/3 - 3s - 884ms/step - loss: 1.4299 - rmse: 1.5700 - val_loss: 0.9829 - val_rmse: 1.6053
Epoch 2/1000
3/3 - 3s - 1s/step - loss: 0.9954 - rmse: 1.3495 - val_loss: 0.9353 - val_rmse: 1.5659
Epoch 3/1000
3/3 - 3s - 1s/step - loss: 0.9453 - rmse: 1.3126 - val_loss: 1.0185 - val_rmse: 1.6341
Epoch 4/1000
3/3 - 2s - 828ms/step - loss: 0.9267 - rmse: 1.2976 - val_loss: 0.8981 - val_rmse: 1.5345
Epoch 5/1000
3/3 - 2s - 778ms/step - loss: 0.8794 - rmse: 1.2691 - val_loss: 0.9391 - val_rmse: 1.5691
Epoch 6/1000
3/3 - 4s - 1s/step - loss: 0.8486 - rmse: 1.2441 - val_loss: 0.9001 - val_rmse: 1.5362
Epoch 7/1000
3/3 - 7s - 2s/step - loss: 0.8204 - rmse: 1.2151 - val_loss: 0.8276 - val_rmse: 1.4730
Epoch 8/1000
3/3 - 7s - 2s/step - loss: 0.7968 - rmse: 1.2007 - val_loss: 0.7634 - val_rmse: 1.4147
Epoch 9/1000
3/3 - 7s - 2s/step - loss: 0.7384 - rmse: 1.1571 - val_loss: 1.0984 - val_rmse: 1.6970
Epoch 10/1000
3/3 - 7s - 2s/step - loss: 0.9073 - rmse: 1.2952 - val_loss: 0.8361 - val_rms